
# Introduction

## Summary

Today, we're going to implementing least squares regression subject to a
*Sobolev smoothness* constraint. That is, we'll talk about this
estimator.

$$
\begin{aligned}
\hat \mu &= \operatorname*{argmin}_{m \in \mathcal{M}^p} \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2 
&& \text{ for } \quad  \mathcal{M}^p = \left\{ m : \lVert m^{(p)} \rVert_{L_2} \le B \right\} 
\end{aligned}
$$

As usual, we'll use `CVXR` to fit our model and the `tidyverse` package for plotting and other tasks.

In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(CVXR)
    CVXR::add_to_solver_blacklist('OSQP')  
})

## Styles

lab.theme = theme(plot.background = element_rect(fill = "transparent", colour = NA),
		    	  panel.background = element_rect(fill = "transparent", colour = NA),
                  legend.background = element_rect(fill="transparent", colour = NA),
                  legend.box.background = element_rect(fill="transparent", colour = NA),
                  legend.key = element_rect(fill="transparent", colour = NA),
				  panel.grid.major=element_line(color=rgb(0,0,0,.05,   maxColorValue=1)),
	              panel.grid.minor=element_line(color=rgb(0,0,0,.02,   maxColorValue=1)),
		          axis.ticks.x = element_blank(),
		          axis.ticks.y = element_blank(),
		          axis.text.x  = element_text(colour = "#aaaaaa"),
		          axis.text.y  = element_text(colour = "#aaaaaa"),
		          axis.title.x  = element_text(colour = "#aaaaaa"),
		          axis.title.y  = element_text(colour = "#aaaaaa", angle=90))
theme_set(lab.theme)



## Fourier Series and Approximation 

Recall the Fourier series characterization of this model we worked out in the [Sobolev Regression lecture](https://machine-learning-theory.github.io/assets/lectures/sobolev-regression-lecture.pdf).

$$ 
\begin{aligned}
\mathcal{M}^p &= \left\{ m(x) = \sum_{j = 0}^\infty m_j \phi_j(x) : \sum_{j=0}^\infty \lambda_j^p m_j^2 \le B^2 \right\} \\
& \text{ for } \phi_{j}(x) = \sqrt{2}\cos(\pi j x) \ \text{ and } \ \lambda_{j}=\pi^2 j^2. 
\end{aligned}
$$ 
There are a few computational tricks we can use to fit this model,
but today we're going to use finite dimensional approximation.


Consider a model that differs only in that we exclude terms for $j \ge N$,

$$ \mathcal{M}^{p,N} = \left\{ m(x) = \sum_{j = 0}^{N-1} m_j \phi_j(x) : \sum_{j=0}^{N-1} \lambda_j^p m_j^2 \le B^2 \right\}. 
$$

This is what we're actually going to fit. To have confidence that it
makes sense to fit the finite-dimensional model $\mathcal{M}^{p,K}$ but
still think as if we've fit the infinite-dimensional model
$\mathcal{M}^p$, we'll want to make sure that this model contains an accurate
approximation to every curve in $\mathcal{M}^p$. That's what Exercises 3 and 4
from the [homework on Sobolev models and finite-dimensional approximation](https://machine-learning-theory.github.io/assets/homework/sobolev-models-homework.pdf) are about. Let's review them.

## Bounding Approximation Error 

### Non-Uniform Approximation
Let's start with non-uniform approximation. If we have a function $m(x)=\sum_{j=0}^{\infty} m_j \phi_j(x)$ and we truncate the series after $N$ terms, we get an approximation $m_N(x)=\sum_{j=0}^{N-1} m_j \phi_j(x)$. Exercise 3 asks you to prove that this is the best approximation to $m$ spanned by $\phi_0, \ldots, \phi_{N-1}$ and calculate the error of this approximation. Let's do both at once by looking at the squared error of an arbitrary function $a_N = \sum_{j=0}^{N-1} a_j \phi_j$ spanned by this truncated basis.
$$
\begin{aligned}
\left\lVert m - a_N \right\rVert^2 
&= \left\lVert \sum_{j=0}^{\infty} m_j \phi_j - \sum_{j=0}^{N-1} a_j \phi_j \right\rVert^2 \\
&= \left\lVert \sum_{j=0}^{N-1} (m_j - a_j) \phi_j + \sum_{j=N}^{\infty} m_j \phi_j \right\rVert^2 
\end{aligned}
$$
We can simplify this by observing that, because our basis functions are orthonormal, the squared norm of a linear combination of them is just the sum of the squared coefficients.
$$
\begin{aligned}
\left\lVert \sum_{j=0}^{\infty} b_j \phi_j \right\rVert^2 
&= \left\langle \sum_{j=0}^{\infty} b_j \phi_j, \ \sum_{k=0}^{\infty} b_k \phi_k \right\rangle \\
&= \sum_{j=0}^{\infty}\sum_{k=0}^{\infty} b_j b_k \left\langle \phi_j, \phi_k \right\rangle \\
&= \sum_{j=0}^{\infty} b_k^2 \quad \text{because} \quad \left\langle \phi_j, \phi_k \right\rangle = \begin{cases} 1 & \text{ if } j=k \\ 0 & \text{ otherwise } \end{cases}
\end{aligned}
$$
Applying this to our approximation error, we get this.
$$
\begin{aligned}
\left\lVert m - a_N \right\rVert^2 
&= \sum_{j=0}^{N-1} (m_j - a_j)^2 + \sum_{j=N}^{\infty} m_j^2 
\end{aligned}
$$
When we use the truncated fourier series, i.e. when we take $a_j = m_j$ for $j=0 \ldots N-1$,
this error is $\sum_{j=N}^{\infty} m_j^2$. And because our error for arbitrary $a_0 \ldots a_{N-1}$ is the sum of this quantity and another sum of squares, we can do no better.

### Uniform Approximation

How big does this approximation error get for functions $m \in \mathcal{M}^p$ to approximate? For the sake of intuition, 
let's start by looking for the hardest 'one-term function' $m_j \phi_j \in \mathcal{M}^p$ to approximate. We can approximate the ones with $j < N$ perfectly, so it has to be one of the others. And among these, it's the one with the largest coefficient $m_j$. Because the model dictates that the coefficients satisfy $m_j^2 \lambda_j^{p} \le B^2$ (i.e. $\lvert m_j\rvert \le \lambda_j^{-p/2} B$), that's the one where $\lambda_j$ is smallest, and because the sequence $\lambda_0, \lambda_1, \ldots$ is increasing, that's $\lambda_{N}$. The maximal error over all 'one-term functions' is $\lambda_j^{-p/2} B$.

This turns out to be the maximal error for all functions $m \in \mathcal{M}^p$, too. Thinking of the sum $\sum_{j=N}^{\infty}m_j^2$ as the dot product between two sequences $\lambda_N^p m_N^2, \lambda_{N+1}^p m_{N+1}^2, \ldots$ and $\lambda_{N}^p, \lambda_{N+1}^p, \ldots$, we can prove it using Hölder's inequality.
$$
\begin{aligned}
\sum_{j=N}^{\infty} m_j^2 
&= \sum_{j=N}^{\infty} \lambda_j^{p} m_j^2 \times \lambda_j^{-p}  \\
&= \left\langle \lambda_N^p m_N^2, \lambda_{N+1}^p m_{N+1}^2, \ldots \ , \ \lambda_{N}^{-p}, \lambda_{N+1}^{-p}, \ldots  \right\rangle_2 \\
&\le \left\lVert \lambda_N^p m_N^2, \lambda_{N+1}^p m_{N+1}^2, \ldots \right\rVert_1 \times \left\lVert \lambda_N^{-p}, \lambda_{N+1}^{-p}, \ldots \right\rVert_\infty \\
&= \sum_{j=N}^{\infty} \lambda_j^{-p} m_j^2 \lambda_j^{p} \times \max_{j \ge N} \lambda_j^{-p} \\
&\le B^2 \times \lambda_N^{-p} 
\end{aligned}
$$

If we want to ensure this maximal error is less than $\epsilon$, then we want to choose $N$ so that $\lambda_N^{-p} \le \epsilon^2/B^2$. In the specific case we're considering here, in which $\lambda_j = \pi^2 j^2$, this means we want to choose
the smallest $N$ with $N^{-2p} \le \pi^{2p} \epsilon^2/B^2$ or equivalently the smallest $N \ge \pi^{-1} (B/\epsilon)^{1/p}$. 

## Implementation

Use CVXR to implement the approximate Sobolev regression, 
$$ 
\hat\mu = \operatorname*{argmin}_{m \in \mathcal{M}^{p,N}} \frac{1}{n}\sum_{i=1}^n \{Y_i - m(X_i)\}^2 
$$
choosing $N$ so that maximal approximation error is less than $\epsilon=.001$. 

Here's a template. In it, I use the function `outer` to compute a matrix $\Phi$ with $\Phi_{ij} = \phi_j(X_i)$. You'll want to make some changes in lines 2-13 and 26. 

In [ ]:
sobolevreg = function(X,Y, p=1, B=1, epsilon=.001){
  N = ceiling((1/pi)*(B/epsilon)^(1/p))
  Phi = outer(X, 0:(N-1), function(x,j) cos(pi*j*x)) 
  lambda = pi^2 * (0:(N-1))^(2*p)
  # specify the parameters and constraints.
  b = Variable(N)
  m = Phi %*% b
  mse = sum((Y - m)^2)/length(Y) 
  constraints = list(sum(lambda * b^2) <= B^2)

  # solve and extract the solution
  solved = solve(Problem(Minimize(mse), constraints))
  beta.hat = solved$getValue(b)
  
  # package up the results in a model object and return it
  model = list(beta.hat=beta.hat, p=p, input=list(X=X, Y=Y)) 
  attr(model, "class") = "sobolevreg"
  model
}

predict.sobolevreg = function(model, newdata=model$input) { 
  x = newdata$X
  beta.hat = model$beta.hat
  N = length(beta.hat)
  Phi.x = outer(x, 0:(N-1), function(x,j) cos(pi*j*x)) 
  
  Phi.x %*% beta.hat 
}

Here's a quick test.

In [ ]:
mu = function(x) { x^2*sin(pi*x) }
X = seq(0,1,by=.01)
Y = mu(X) + .1*rnorm(length(X))

model1 = sobolevreg(X,Y,p=1, B=1)
model2 = sobolevreg(X,Y,p=2, B=1)
ggplot() + 
    geom_line(aes(x=X, y=mu(X)), color='black') + 
    geom_point(aes(x=X, y=Y), alpha=.2) + 
    geom_line(aes(x=X, y=predict(model1)), color='blue') + 
    geom_line(aes(x=X, y=predict(model2)), color='red')  

## Comparison

Let's take a minute to build our intuition about what Sobolev smoothness
means by comparing the Sobolev seminorm to others we've been using.
Consider the total variation, ($p=1$) Sobolev, and Lipschitz seminorms
on functions on $[0,1]$. 
$$ 
\rho_{TV}(m) = \int_0^1 \lvert m'(x) \rvert dx, \quad 
\rho_{-\Delta}(m) = \sqrt{\int_{0}^1 \lvert m'(x)\rvert^2 dx}, \quad 
\rho_{Lip}(m) = \max_{x \in [0,1]} \lvert m'(x)\rvert.  
$$

::: exercise
1.  What are these for the polynomial functions $m(x)=x^k$?
2.  Name a function for which these are all equal.
3.  Name a function for which none of these are finite.
4.  Name a function for which the first is finite and the others are
    not.
5.  Name a function for which the first two are finite and the last is
    not.
:::

Go ahead and swap these signals into the block above and see how well the Sobolev model fits them.
Try out different values of $p$ and $B$ to see how it changes the fit. And compare to Lipschitz and/or bounded variation regression, too.

Here are a couple more signals to try.

In [ ]:
## polynomial of order k normalized to have p=1 sobolev norm=1
mu = function(x,k=2) { abs(x)^k * sqrt(2*k-1)/k } 
## cosine of period 2/k normalized to have p=p sobolev norm=1
mu =  function(x, k=3, p=2) { sqrt(2)*cos(pi*k*x)/(pi * k)^p }
## step function
mu = function(x) {  .5*(x > 1/3) + .4*(x > 2/3) }
## step line
mu = function(x) { (x-.5)*(x >= .5) }
## damped high-frequency oscillations near 0
mu = function(x) { x*sin(pi/x)/2 }

To do our comparisons, we'll need regression code for these other methods. Here's code copied from previous labs and homeworks. 

In [ ]:
## A function we'll want. unique with an inverse mapping.

invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

a = c(1,2,3,3,4,5,5,5,6)
unique.a=invert.unique(a)
stopifnot(a[unique.a$elements][unique.a$inverse] == a)

## Fitting Functions

monotonereg = function(X, Y, decreasing = FALSE) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list(if (decreasing) { diff(m) <= 0 } else { diff(m) >= 0 })

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.monotonereg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, input = input)
  attr(model, "class") = "monotonereg"
  model
}

convexreg = function(X, Y, concave = FALSE, monotone = NULL) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  # Interpretation (rearrange): secant slopes are increasing
  uX = X[unique.X$elements]
  ii = 1:(n-2)
  constraints = 
    list(((m[ii+1]-m[ii])  * (uX[ii+2]-uX[ii]) - 
          (m[ii+2]-m[ii]) *  (uX[ii+1]-uX[ii])) * (-1)^concave <= 0)
  if(!is.null(monotone)) { 
       decreasing = monotone == 'decreasing'
       constraints = c(constraints, diff(m) * (-1)^decreasing >= 0)
  }
  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.convexreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, input = input)
  attr(model, "class") = "convexreg"
  model
}

bvreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list( sum(abs(diff(m))) <= B )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.bvreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "bvreg"
  model
}

lipreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  uX = X[unique.X$elements]
  constraints = list( abs(diff(m)) <= B * diff(uX) )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.lipreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "lipreg"
  model
}

## Prediction Functions

# make predictions based on piecewise-constant interpolation
# we use the curve that jumps at each observation and is otherwise constant
# that is, if X[1] < X[2] < ..., 
#   mu.hat(x) for x between X[k] and X[k+1] is mu.hat(X[k])   [case 1]
#             for x > X[k]  is mu.hat(X[k])                   [case 2]
#             for x < X[1]  is mu.hat(X[1])                   [case 3]
predict.piecewise.constant = function(model, newdata=data.frame(X=model$input$X)) {
  Y = model$mu.hat; X=model$X; x=newdata$X
  # for each new data point x in newdata$X, 
  # find the closest observed X[k] left of x
  # i.e., the largest k for which X[k] <= x 
  # this covers cases 1 and 2
  # i will be a vector of these numbers k, with one for each x in newdata$X
  i = findInterval(x, X) 
  # if there is no X[k] < x, findInterval tells us k=0
  # to cover case 3, we want X[k] for k=1 when this happens.
  i[i==0] = 1
  # report the values of mu.hat(X[k]), one for each x
  Y[i]
}

predict.piecewise.linear = function(model, newdata=data.frame(X=model$input$X)) {
  Y = model$mu.hat; X=model$X; x=newdata$X; n = length(X) 
  # for each new data point x[k]
  # find the closest observed X[i[k]] left of x[k]
  # i.e., i[k] is the largest integer i for which X[i] <= x[k] 
  i = findInterval(newdata$X, X) 
  # If there is no X[i] < x[k], findInterval sets i[k]=0
  #  and we'll want to act as if we'd gotten 1 so we use the
  #  line through (X[1], Y[1])  and (X[2], Y[2])
  # If that k is n, we'll want to act as if we'd gotten n-1 so we use 
  #  the line through (X[n-1], Y[n-1])  and (X[n], Y[n])
  i[i==0] = 1; i[i==n] = n-1
  # make a prediction using the formula y - y0 = (x-x0) * slope 
  Y[i] + (x-X[i]) * (Y[i+1]-Y[i])/(X[i+1]-X[i])
}

predict.monotonereg = predict.piecewise.constant
predict.convexreg = predict.piecewise.linear
predict.bvreg = predict.piecewise.constant
predict.lipreg = predict.piecewise.linear

## Conveniences

prediction.function = function(model) { 
  if(class(model)=='function') { model }
  else { function(x) { predict(model, newdata=data.frame(X=x)) } }
}


Now for the comparisons themselves. I'll start with the signal in the 'quick test' block above.

In [ ]:
models = list(bv  = \(X,Y)bvreg(X,Y,B=1),
              lip = \(X,Y)lipreg(X,Y,B=1),
              sobolev1 = \(X,Y)sobolevreg(X,Y, B=1.5, p=1),
              sobolev2 = \(X,Y)sobolevreg(X,Y, B=1, p=2))

plot.predictions = function(X,Y,models) {
  x = seq(-1,1,by=.001)
  predictions = models |> map(function(reg) { 
    model = reg(X,Y)
    muhat.x = predict(model, newdata=data.frame(X=x))
    data.frame(x=x, y=muhat.x)
  }) |> list_rbind(names_to = "model")

  reflected = function(f) { \(x) ifelse(x >= 0, f(x), f(-x)) }
  ggplot() + geom_line(aes(x=x, y=reflected(mu)(x)), alpha=.2, size=1) + 
             geom_point(aes(x=X, y=Y), alpha=.1) + 
             geom_line(aes(x=x, y=y, color=model), alpha=.8, data=predictions) + 
             coord_cartesian(xlim=c(0,1), ylim=range(Y)) + theme(legend.position = "bottom") + 
             labs(x="", y="")
}

mu = function(x) { x^2*sin(pi*x) }
X = seq(0,1,by=.01)
Y = mu(X) + .1*rnorm(length(X))
plot.predictions(X,Y,models)

Here's an interesting one: $\mu(x)=x$. Even though the second derivative $\mu''$ is zero, the second order Sobolev model
doesn't fit it well near $x=0$. You can even get rid of the noise entirely. What's going on?

In [ ]:
mu = function(x) { x }
X = seq(0,1,by=.01)
epsilon = .1*rnorm(length(X))
plot.predictions(X,mu(X)+epsilon,models)
plot.predictions(X,mu(X),models)

It's a reflection thing. When you take a look at what's going on to the left of $x=0$ when we extend $\mu(x)$ by reflection,
it makes a little more sense.

In [ ]:
epsilon = .1*rnorm(length(X))
plot.predictions(X,mu(X)+epsilon,models) + coord_cartesian(xlim=c(-1,1))
plot.predictions(X,mu(X),models) + coord_cartesian(xlim=c(-1,1))